In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt
import tensorflow as tf


from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

In [4]:
dataset = df.values

In [5]:
X = []
for ind in df.index:
    X.append(df.loc[ind].CT)

X = np.array(X)/10
X = X.reshape(X.shape[0], 512, 512, 1)

In [6]:
from sklearn.preprocessing import MinMaxScaler
y = result.values
scaler = MinMaxScaler()
scaler.fit(y)
y_scaled = scaler.transform(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2)

In [8]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(10, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu', 
                 input_shape=(512,512,1), kernel_initializer='random_normal', 
                 bias_initializer='zeros' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='relu'))
# output layer
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 511, 511, 10)      50        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 10)      0         
_________________________________________________________________
flatten (Flatten)            (None, 158760)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [10]:
model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam')

In [13]:
type(X_train[0][0][0][0])

numpy.float64

In [12]:
model.fit(X_train,y_train, batch_size=32, epochs=20, validation_data=(X_test,y_test))

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[32,10,511,511] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-12-3ac1d9fe6060>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_979]

Function call stack:
train_function


In [ ]:
scaler.inverse_transform(model.predict(X_test))

In [ ]:
n=8
example = df.iloc[n].CT.reshape(1,512,512,1)
print(scaler.inverse_transform(model.predict(example)))
print(result.iloc[n])

In [ ]:
abs(scaler.inverse_transform(model.predict(X_test)) - scaler.inverse_transform(y_test)).mean()

In [ ]:
scaler.inverse_transform(y_test)